In [7]:
import requests
import numpy as np
import pandas as pd

In [8]:
# API 
api_key = '6KP4N5HM0UYLK3GP'

# Ticker Merck
symbol = 'MRK'

In [9]:
# Funzione per ottenere informazioni sull'azienda e dividendi annuali
def get_company_info(symbol):
    url_overview = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}'
    response = requests.get(url_overview)
    data = response.json()
    
    print("\nInformazioni di base sull'azienda:")
    for key, value in data.items():
        print(f"{key}: {value}")
    
    return data

In [10]:
# Funzione per ottenere i dividendi storici
def get_dividends(symbol):
    url_dividends = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={symbol}&apikey={api_key}'
    response = requests.get(url_dividends)
    data = response.json()
    
    # Estrggo i dividendi e converto in un DataFrame
    monthly_data = data.get('Monthly Adjusted Time Series', {})
    dividends = pd.DataFrame([(date, float(values['7. dividend amount'])) for date, values in monthly_data.items() if float(values['7. dividend amount']) > 0],
                             columns=['Date', 'Dividend'])
    dividends['Date'] = pd.to_datetime(dividends['Date'])
    dividends.set_index('Date', inplace=True)
    dividends = dividends.resample('Y').sum()  # Raggruppo per anno per ottenere i dividendi annuali
    
    # Calcolo del tasso di crescita
    dividends['Growth'] = dividends['Dividend'].pct_change()
    
    print("\nDividendi annuali e tasso di crescita calcolato:")
    print(dividends)
    
    return dividends

In [11]:
# Ottenere informazioni sull'azienda
info = get_company_info(symbol)

# Ottenere i dividendi storici
dividends = get_dividends(symbol)

# Calcolo del tasso di crescita medio considerando solo gli ultimi 10 anni
if len(dividends) > 10:
    last_10_years_growth = dividends['Growth'].iloc[-10:]  # Prendi solo gli ultimi 10 anni di crescita
else:
    last_10_years_growth = dividends['Growth']

# Calcolo della media escludendo il primo NaN
valid_growth = last_10_years_growth.dropna()  # Rimuove i NaN per calcolare la media correttamente
avg_growth_rate = valid_growth.mean()

print(f"\nTasso di crescita medio dei dividendi (ultimi 10 anni): {avg_growth_rate:.2%}")



Informazioni di base sull'azienda:
Symbol: MRK
AssetType: Common Stock
Name: Merck & Company Inc
Description: Merck & Co. is an American multinational pharmaceutical company headquartered in Kenilworth, New Jersey. It is named after the Merck family, which set up Merck Group in Germany in 1668.
CIK: 310158
Exchange: NYSE
Currency: USD
Country: USA
Sector: LIFE SCIENCES
Industry: PHARMACEUTICAL PREPARATIONS
Address: 2000 GALLOPING HILL ROAD, KENILWORTH, NJ, US
OfficialSite: https://www.merck.com
FiscalYearEnd: December
LatestQuarter: 2024-06-30
MarketCapitalization: 258246443000
EBITDA: 31673999000
PERatio: 21.31
PEGRatio: 0.0685
BookValue: 17.19
DividendPerShare: 3.04
DividendYield: 0.0302
EPS: 4.78
RevenuePerShareTTM: 24.95
ProfitMargin: 0.192
OperatingMarginTTM: 0.265
ReturnOnAssetsTTM: 0.114
ReturnOnEquityTTM: 0.334
RevenueTTM: 63174001000
GrossProfitTTM: 42077000000
DilutedEPSTTM: 4.78
QuarterlyEarningsGrowthYOY: -0.333
QuarterlyRevenueGrowthYOY: 0.044
AnalystTargetPrice: 134.6
An

In [13]:
# Prendo l'ultimo dividendo pagato
last_dividend = dividends['Dividend'].iloc[-1]

# Assumo di un tasso di rendimento richiesto
required_rate_of_return = 0.05
print(f"\nUltimo dividendo pagato: ${last_dividend:.2f}")
print(f"Tasso di rendimento richiesto: {required_rate_of_return:.2%}")


Ultimo dividendo pagato: $2.31
Tasso di rendimento richiesto: 5.00%


In [14]:
# Proiezione dei dividendi per i prossimi 10 anni
future_dividends = []
for i in range(1, 11):
    future_dividend = last_dividend * ((1 + avg_growth_rate) ** i)
    future_dividends.append(future_dividend)
    print(f"Dividendo previsto per l'anno {i}: ${future_dividend:.2f}")


Dividendo previsto per l'anno 1: $2.38
Dividendo previsto per l'anno 2: $2.46
Dividendo previsto per l'anno 3: $2.54
Dividendo previsto per l'anno 4: $2.62
Dividendo previsto per l'anno 5: $2.70
Dividendo previsto per l'anno 6: $2.78
Dividendo previsto per l'anno 7: $2.87
Dividendo previsto per l'anno 8: $2.96
Dividendo previsto per l'anno 9: $3.05
Dividendo previsto per l'anno 10: $3.15


In [15]:
# Calcolo del valore attuale dei dividendi futuri
discounted_dividends = [div / ((1 + required_rate_of_return) ** i) for i, div in enumerate(future_dividends, start=1)]

print("\nDividendi attualizzati:")
for i, div in enumerate(discounted_dividends, start=1):
    print(f"Anno {i}: ${div:.2f}")


Dividendi attualizzati:
Anno 1: $2.27
Anno 2: $2.23
Anno 3: $2.19
Anno 4: $2.15
Anno 5: $2.11
Anno 6: $2.08
Anno 7: $2.04
Anno 8: $2.00
Anno 9: $1.97
Anno 10: $1.93


In [16]:
# Calcolo del valore terminale al decimo anno
terminal_value = future_dividends[-1] * (1 + avg_growth_rate) / (required_rate_of_return - avg_growth_rate)
discounted_terminal_value = terminal_value / ((1 + required_rate_of_return) ** 10)

print(f"\nValore terminale al decimo anno: ${terminal_value:.2f}")
print(f"Valore terminale attualizzato: ${discounted_terminal_value:.2f}")


Valore terminale al decimo anno: $175.54
Valore terminale attualizzato: $107.77


In [18]:
# Calcolo del valore teorico dell'azione
theoretical_value = sum(discounted_dividends) + discounted_terminal_value

print(f"\nValore teorico dell'azione di {symbol} (DDM): ${theoretical_value:.2f}")


Valore teorico dell'azione di MRK (DDM): $128.74


In [20]:
# Scaricare il prezzo attuale di Merck
url_price = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={api_key}'
response = requests.get(url_price)
current_price = float(response.json()['Global Quote']['05. price'])

print(f"\nPrezzo attuale di {symbol}: ${current_price:.2f}")


Prezzo attuale di MRK: $101.88
